# Lab 2

We will be working through a few problems so you can see how these attacks / vulnerabilities work

In [22]:
####################################################
# The following functions are written and functional 
# They are for your use
####################################################

# DECODE FUNCTION
# function to take decimal value message and convert to ascii plaintext
def decode_message(message):
    m = int(message)
    length = math.ceil(m.bit_length() / 8)
    decoded_message = m.to_bytes(length, byteorder='little').decode()
    return decoded_message

# HASH FUNCTION
# function takes a string message and returns hexadecimal hash
def hash_message(message):
    import hashlib                        # gives us access to sha256()
    hash = hashlib.sha256()               # instantiates a sha256 object
    hash.update(message.encode('UTF-8'))  # creates hash of the message
    return hash.hexdigest()               # returns hexadecimal value

## **TODO List**  
**_PART 1:  Break RSA factoring_**  
PART 2:  Break RSA with Square Roots & CRT  
PART 3:  Rabin Signatures  
PART 4:  Shamir Secret Sharing

This should be a straight forward exercise for you by now, so consdier this review.  
How can we break RSA simply by factoring...

For starters we need to factor a large number.  
We will not use overly large numbers here.  
They will be reasonable to work with in a jupyter notebook.

First step is to factor.  
While we could write a brute force style factoring algorithm,  
that would take a long time to process on a single personal computer.  
It is instructiver to think about the problem though, and I encourage the thought.  
There are many ways to make a factoring algorithm more efficient.  

For now, use any method you'd like to find the factors of N.   

In [3]:
################
# DATA DATA DATA
################


# Aaron's Public Key (E, N)
E = 10492451
N = 323777872629222912584202035158155151876683308819


# intercepted message text
C1 = 33013487164610291241411888177625215947161012926
C2 = 151308661650694256002121415408590077507068349844

In [13]:
############################
# Question 1: Your work here
############################

# find p and q, use qsieve to factor
factors = N.factor(algorithm="qsieve") 

#call, assign tuple values
p = factors[0][0] 
q = factors[1][0]

#check if prime
print(p.is_prime()) #
print(q.is_prime())

print("p:\t" + str(p))
print("q:\t" + str(q))

/opt/sagemath-9.1/local/lib/python3.7/site-packages/ipykernel/zmqshell.py:537: RuntimeWarning: the factorization returned by qsieve may be incomplete (the factors may not be prime) or even wrong; see qsieve? for details
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


True
True
p:	355718838765115951494559
q:	910207268620417601264141


In [15]:
############################
# Question 2: Your work here
############################

# now that you know p, q what is phi(N)
# you can use the function `euler_phi(N)` to check your work

phi_N = (p-1)*(q-1)

print(phi_N == euler_phi(N))

print("phi_N:\t" + str(phi_N))

True
phi_N:	323777872629222912584200769232047766343130550120


Since we now have access to _phi_N_, we need only find the multiplicative inverse of the encryption key.  
Use the `inverse_mod()` function here.

In [17]:
############################
# Question 3: Your work here
############################

# What is the decryption exponent D?

D = inverse_mod(E,phi_N)

print("D:\t" + str(D))

D:	42229904065223046877996389967069189172752452971


You have the decryption exponent now...  
The rest is up to you, what does the message C1 say?

In [29]:
############################
# Question 4: Your work here
############################

# After C1 has been decoded and decrypted, 
# what is the plaintext message?

M1 = power_mod(C1,D,N)
print("message C1: \n\t" + str(decode_message(M1)))


message C1: 
	Exfil immediately: 
message C2: 
	RV: 12S XF 7394 9654


## **TODO List**  
PART 1:  Break RSA factoring  
**_PART 2:  Break RSA with Square Roots & CRT_**  
PART 3:  Rabin Signatures  
PART 4:  Shamir Secret Sharing

We will now break C2 using a different tactic, the square root method!  

We still have access to the same public key, so we should find square roots of a number _mod N_  
You can accomplish this using a brute force method like we had in the notes, by checking every value.  

But we can do better.  We know that 1 has a square root pair of {1, -1} or here {1, N-1}  
So let's start by attacking the value 1.  
We would square every value until we get an answer which is equal to _1 mod N_, that would give us enough information to continue.  

Here, we will use the wonderful code `sqrt()` to help us out.

In [37]:
# here are the square roots of 1 mod N
[r1, r2, r3, r4] = mod(1,N).sqrt(all=True)
print("The square roots of 1 mod N are:")
print("\troot 1: " + str(r1))
print("\troot 2: " + str(r2))
print("\troot 3: " + str(r3))
print("\troot 4: " + str(r4))


############################
# Question 5: Your work here
############################


# check that all 4 of the values are indeed square roots of 1 mod N

# check r1
print(r1**2 % N)
# check r2
print(r2**2 % N)
# check r3
print(r3**2 % N)
# check r4
print(r4**2 % N)


The square roots of 1 mod N are:
	root 1: 1
	root 2: 157801397976567248731216116353265405817139091503
	root 3: 165976474652655663852985918804889746059544217316
	root 4: 323777872629222912584202035158155151876683308818
1
1
1
1


we are going to use the sublime formula `gcd(r1 + r2, N)` to find a factor  
to do this we need let's take the first two square roots, r1, r2  

_Note_ This sublime formula is a direct result of our study of the Chinese Remainder Theorem

In [49]:
############################
# Question 6: Your work here
############################

# find p

p = gcd(r1+r2, N)

print("p:\t" + str(p))

p:	355718838765115951494559


In [55]:
############################
# Question 7: Your work here
############################

# what is q?
q = gcd(r2+r4,N)

print(p*q == N) #check that the result is q, and that product is N

print("q:\t" + str(q))

True
q:	910207268620417601264141


Now we have _p_ and _q_, so we can now find D, the decryption exponent and the next part of the message

In [74]:
############################
# Question 8: Your work here
############################

phi_N = ((int(p-1))*int((q-1)))
print(phi_N)

# what is the broken message, C2?
D = inverse_mod(E,phi_N)
M2 = decode_message(int(power_mod(C2, D, N)))

print("message C2: \n\t"+ str(M2))

# what is the entire message, C1 + C2, then?
print("entire message: \n\t" + str(decode_message(M1)) + str(M2))

323777872629222912584200769232047766343130550120
message C2: 
	RV: 12S XF 7394 9654
entire message: 
	Exfil immediately: RV: 12S XF 7394 9654


## **TODO List**  
PART 1:  Break RSA factoring  
PART 2:  Break RSA with Square Roots & CRT  
**_PART 3:  Rabin Signatures_**  
PART 4:  Shamir Secret Sharing

Now that we are more familiar with square roots in a modulus, let's look at how a rabin signature can work. 
In this lab we used to go into a lot of detail about how difficult it is to process certain tasks.  
For instance, factoring large numbers is incredibly difficult, and we can run a brute force algorithm one after another on increasingly large N to see how the time requirements increase exponentially.  

In light of that difficulty Rabin signatures are a breeze to verify.  
If you work with outlook and signed email frequently, you'll notice a distinct user experience horror when trying to open a signed email, as the process of validating that email works too slowly.

In [76]:
# Aaron's message to sign
message = "The Women's World Cup was won in dramatic fashion by Spain.   EOF   padding:00000004"

We will need a new N value, since the N value from above is small, maybe 120 bits long or so.  
Here we know that this hash is 256 bits long, so we need an N which is at least as big.  
Let's check that!

In [77]:
# hash the message, the basis of any digital signature
message_hash = hash_message(message)
print("hash of the message: " + message_hash)
print()

# start creating the signature
p = 318673169171047839116951099424178531399
q = 288237402043873380399913216798519703159
rabin_N = p * q

print("           Size of N in bits: " + str(N.nbits()))
print("Size of message_hash in bits: " + str(Integer(int(message_hash,16)).nbits()))
print("     Size of rabin_N in bits: " + str(rabin_N.nbits()))

hash of the message: 81e385146ed42214d50bff181a50636aaa7467ba7508f59451bb700b0f7344db

           Size of N in bits: 158
Size of message_hash in bits: 256
     Size of rabin_N in bits: 256


We will create a rabin signature now...  
We want the hash to be a perfect square, since the Rabin signature is it's square root.

Start with the hash, and turn it into an integer.  
Check if that integer is a perfect square.  
If not, then increment the value and recheck.  

_note: In practice a protocol needs to be devised on how to add padding.  
Remember the padding is required in Rabin signatures in order to allow for incrementing_ 

our path here:
1. we are trying to determine the square root of the int_hash, so we can send our signature
2. finding square roots is difficult, to make it quicker we will use the CRT, yay!
3. we will find the square root _mod p_  and the square root _mod q_
4. then we will combine these using the CRT, which will give us a square root _mod rabin_N_

In [78]:
# this should all look familiar from lab 1
int_hash = int(message_hash, 16)

# find a square root of the hash mod p
int_hash_modp = mod(int_hash, p)

# is there a square root mod p?
sqrt_modp = mod(int_hash_modp, p).sqrt()
print(sqrt_modp)

42651880768614775674104685552082528197


The answer was an integer value so we are set!
We know there is a square root  

If there weren't an integer value then we would have to go back and pad the message.  
Once you've added padding you could retry these calculations until you get an answer.  

I did this manually, if you want to check for yourself.  
You can see that a padding:00000000 doesn't yield a square root in p  
You can see that a padding:00000001 doesn't yield a square root in p  
You can see that a padding:00000002 does yield a square root in p, but not q  
... a padding:00000005 yielded both

Since we have an answer, we will continue and validate that we also have a square root mod q.  
(if there isn't we need to add padding and retry until we get valid roots in both p and q)

In [79]:
# find a square root of the hash mod q
int_hash_modq = mod(int_hash, q)

# is there a square root mod p?
sqrt_modq = mod(int_hash_modq, q).sqrt()
print(sqrt_modq)

79872524758206451134704322268869453550


Yay! this works.  
The last step to creating the rabin signature is using the CRT to combine these two values.  
The first value is the square root of our int_hash mod p,  
the second value is the square root of our int_hash mod q,  
so combining them using CRT will give us the square root of our int_hash mod p*q!

In [80]:
sqrt = crt(Integer(sqrt_modp), Integer(sqrt_modq), p, q)
print("The square root of int_hash mod rabin_N: \n\t" + str(sqrt))

The square root of int_hash mod rabin_N: 
	27350518179012306423506070687590633174921200541952403272620345755928206924762


We finally have our signature!  
Depending on your level of familiarity with programming, this may have seemed like a lot of work.  
It really was just a few lines of code, and we've calculated quickly.  

We did good work! Let's send our message to the world!

In [81]:
# Aaron's signed message!!!
signed_message = "The Women's World Cup was won in dramatic fashion by Spain.   EOF   padding:00000004"
signature = 27350518179012306423506070687590633174921200541952403272620345755928206924762

# Aaron's public signing key
N = rabin_N

Now you as a class, can validate my signature!

In [82]:
############################
# Question 9: Your work here
############################

# create your own hash of my message
class_hash = hash_message(signed_message)
print("class hash: \t" + str(class_hash))

# recreate the hash from the signature
# square the signature (don't forget the modulus)
sig_squared = (signature**2) % N

# cast back into a hex
print("signed hash: \t" + str(hex(sig_squared)[2:]))

# do they match???
# is the signature valid???

class hash: 	81e385146ed42214d50bff181a50636aaa7467ba7508f59451bb700b0f7344db
signed hash: 	81e385146ed42214d50bff181a50636aaa7467ba7508f59451bb700b0f7344db


## **TODO List**  
PART 1:  Break RSA factoring  
PART 2:  Break RSA with Square Roots & CRT  
PART 3:  Rabin Signatures  
**_PART 4:  Shamir Secret Sharing_**

Finally a quick demo of Shamir Secret sharing.  
Remember the idea is to use a polynomial of a certain order to share a secret.  
We will have a quorum, which is one more than the order of the polynomial.  
Then we will be able to share the secret with as many people as we want.  

The secret will be the value of the polynomial evaluated at zero.  
$f(0) = $ _secret_

Step 1: determine the quorum  
answer: say the quorum is 5 for this exercise  

Step 2: determine the polynomial with modulus
answer: randomly decide the polynomial is...  
$f(x) = 15x^4 + 27x^3 + 2x^2 + 19x + secret$ _mod p_ 

Step 3: determine the secret
answer: let's pretend we are looking for grid coordinates for an underground nuclear facility.  
These coordinates will come as a set of eight digits for a given military map.  
This is using the standard MGRS, military grid reference system.  
So, _secret = [redacted]_  

Step 4: distribute keys  
evaluate f(1), f(2), f(3)... and distribute one pair of (x, y) coordinates to each member.  

As secret keepers we don't disclose the coefficients of the polynomial.  
These are sacrisanct, and in fact we shouldn't ever know what they are individually.  
Shamir secret sharing relies on the fact that if the correct number of people come together,  
each with valid keys, they can recreate the polynomial and learn the secret.  

In [83]:
# five people come together and they each have keys
key_1 = [1,78958221]
key_2 = [2,106365037]
key_3 = [3,157748887]
key_4 = [4,247189185]
key_5 = [5,110892620]

# also we all know the modulus p
p = 282322877

How can the 5 people recreate the polynomial?  
they need to identify the 5 coefficients!  

Since this is a polynomial of order 4:  
$f(x) = a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0$  

We need to identify what the values: $a_4, a_3, a_2, a_1, a_0$ are...  
namely the value $a_0$ gives us the SECRET!!!

Do you remember in algebra when you solved a system of 2 equations and 2 unknowns?  
$2x + 5y = -10$  
$3x + 10y = 6$  

You can multipy the first equation by 2 and subtract the second equation:  
$4x + 10y = -20$  
$3x + 10y = 6$  

yielding: $x + 0y = -26$  
so we soloved that $x = -26$ and $y = 8.4$  


That is what we will set up here.  
The only difference is we will use a system of 5 equations and 5 unknowns.

Look at this function:  
$f(x) = a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0$  

take the keys and you can substitute the values of x, y from the keys.  
_key_1_: $78958221 = a_4 + a_3 + a_2 + a_1 + a_0$  
_key_2_: $106365037 = 16a_4 + 8a_3 + 4a_2 + 2a_1 + a_0$  
_key_3_: $157748887 = 243a_4 + 81a_3 + 9a_2 + 3a_1 + a_0$  
_key_4_: $247189185 = 256a_4 + 64a_3 + 16a_2 + 4a_1 + a_0$  
_key_5_: $110892620 = 625a_4 + 125a_3 + 25a_2 + 5a_1 + a_0$  

The coefficients _a_ are now the 'unknowns' for which we will solve.

In [84]:
# start by declaring a finite Ring... forces modular arithmetic
R = IntegerModRing(p)

# create matrix using the coefficients from each row above
matrix = []
matrix.append([1, 1, 1, 1, 1])
matrix.append([16, 8, 4, 2, 1])
matrix.append([81, 27, 9, 3, 1])
matrix.append([256, 64, 16, 4, 1])
matrix.append([625, 125, 25, 5, 1])

# combine Ring and matrix
ring_matrix = Matrix(R, matrix)

# make vector for solutions
vect = []
vect.append(78958221)
vect.append(106365037)
vect.append(157748887)
vect.append(247189185)
vect.append(110892620)

# combine Ring and vector
ring_vector = vector(R, vect)

# let's solve this thing...
solution = ring_matrix.solve_right(ring_vector)
print(solution)
print()
print("The secret is: " + str(solution[4]))

(185423, 492339, 4398908, 7982374, 65899177)

The secret is: 65899177


The answer we want is $a_0$ which is the last entry in our solution  
If you are feeling industrious you'll remember that our secret is the coordinates to the secret underground nuclear facility...  

You are operating in the vicinity of map `10S EG`,
if you want to add the 8 digit solution to this map code like `10S EG XXXXXXXX`, then you can go to this website and see where the secret facility is located.  

https://legallandconverter.com/p50.html  
use the top link "MGRS Grid Reference Search"  
and select map of your choice

In [ ]:
############################
# Question 10: Your work here
############################

# where is the nuclear facility?

Center Field, California Memorial Stadium in Berkeley

